- Get top songs from Billboard top 100
- Use Spotify API to get Audio features and time for all the songs
      - Look at trends over the years
- Use Genius API to get lyrics of all the songs
- Use NLP (bag of words maybe?) to see patterns in lyrics over the years; 
     -maybe make a word cloud
     -% of unique words over the years
- Shiny App?
     
     

In [19]:
from requests import get
import pandas as pd
import re
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from time import time
from warnings import warn
from IPython.core.display import clear_output
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Web Scraping Billboard Top 100 (1950-2009)

Initial years have 30-80 top songs /year
' gets parsed as â€™  - did a find & replace in excel

In [9]:
#Testing Connection
response=get("http://billboardtop100of.com/1975-2/")
soup=BeautifulSoup(response.content,"html.parser")
soup.title
musicdata=soup.find_all("tr")
print(type(musicdata))
print(len(musicdata))

<class 'bs4.element.ResultSet'>
100


In [ ]:
#Getting Data
years_url = [str(i) for i in range(1950,2010)]
years=[]
ranks=[]
artists=[]
songs=[]


# Preparing the monitoring of the loop
start_time = time()
req = 0

# For every year in the interval 1940-2016
for year_url in years_url:

    # Make a get request
    response = get("http://billboardtop100of.com/"+ year_url +"-2/")
    

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    req += 1
    elapsed_time = time() - start_time
    print('Req:{}; Frequency: {} req/s'.format(req, req/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Req: {}; Status code: {}'.format(req, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if req > 7700:
        warn('Number of requests was greater than expected.')  
        break 

        # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup.title)
        # Select all the 100 songs from each page
    musicdata = soup.find_all("tr")
    
    for data in musicdata:
        year=soup.find("h1",class_="entry-title").text
        years.append(year)
    
        rank=data.contents[1].text
        ranks.append(rank)
     
        artist=data.contents[3].text
        artists.append(artist)
    
        song=data.contents[5].text
        songs.append(song)
         

<title>1951 - Billboard Top 100</title>


In [ ]:
#Saving Data to DF
df = pd.DataFrame({'year':years,
                   'rank':ranks,
                   'artist':artists,
                   'song':songs})
print(df.info())
df
df.to_csv('Top.csv')

In [21]:
#Saving Data to csv file
data=pd.read_csv("top.csv")
data1=data.iloc[0:1000]


# Using Spotify API (through spotipy) to get audio features  (Incomplete)

In [23]:
with open('SpotifyID.txt', 'r') as f:
    client_id = f.read().replace('\n','')
with open('SpotifySecret.txt', 'r') as f:
    client_secret = f.read().replace('\n','')

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False
count=0
uris=[]
for i in range(0,10):
    artist=data1.loc[i,"artist"]
    title=data1.loc[i,"song"]

    search_querry = title + ' ' + artist
    result = sp.search(search_querry)
    
    if(result["tracks"]["items"]==[]):
        search_querry2 = title
        result2 = sp.search(search_querry2)
        
        
        if(result2["tracks"]["items"]==[]):
            count=count+1
            uri="error"
            print("error",count)
        else:
            print(result2["tracks"]["items"][0]["uri"],artist)
            uri=result2["tracks"]["items"][0]["uri"]
    else:
        print(result["tracks"]["items"][0]["uri"],artist)
        uri=result["tracks"]["items"][0]["uri"]
    
           
            

spotify:track:1fhLgOJgIIZEsWWffk8ljs Gordon Jenkins and The Weavers
spotify:track:3k5ycyXX5qsCjLd7R2vphp Nat King Cole
spotify:track:7rRGujA12UJcRUz7DxUDwQ Anton Karas
spotify:track:39Q20GZHKK0IK8oKkhfxiJ Gary and Bing Crosby
spotify:track:33FdwvfiSBRbE2AQ4UehaR Gary and Bing Crosby
spotify:track:68ZOKozM9xmlp1iwaQSwY7 Teresa Brewer
spotify:track:5uEwpyOYHFGqme4WnVQhfs Guy Lombardo
spotify:track:7Jf323ttHKUnPylFWiaGl3 Red Foley
spotify:track:00vJeips2Rwr2qN1qODvyf Sammy Kaye
spotify:track:0BhbE7LZQ2PpwvEGkrYug0 Sammy Kaye and Don Cornell


# Using Genius API to get lyrics